In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

# Modify Connection

In [59]:
df_total = pd.read_csv('Total List.csv')
df_urls = df_total['Product Link']
for item in df_urls:
    if item == 'NaN':
        item = np.nan
        
df_urls

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
2538         https://www.producthunt.com/posts/timer-plus
2539    https://www.producthunt.com/posts/chat-plus-fo...
2540    https://www.producthunt.com/posts/just-in-case...
2541    https://www.producthunt.com/posts/better-blog-...
2542           https://www.producthunt.com/posts/langsaas
Name: Product Link, Length: 2543, dtype: object

In [140]:
headers = {"Accept": "application/json, text/plain, */*",
           "Accept-Encoding": "gzip, deflate, br",
           "Accept-Language": "en-US,en;q=0.9", "Connection": "keep-alive",
           "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Mobile Safari/537.36"}

attr = {
    'product_title': 'h1',
    'class_title': 'text-18 sm:text-32 md:text-32 font-bold text-dark-gray',
    'team': 'a',
    'class_team': 'text-14 sm:text-16 font-semibold text-light-gray py-3',
    'product_category': 'span',
    'class_category': 'text-12 font-semibold text-dark-gray text-dark-gray',
    'total_up_votes': 'div',
    'class_up_votes': 'text-14 font-semibold text-white uppercase',
    'class_comments': '#about > div.mb-10.flex.flex-row.justify-between > div:nth-child(3) > div.text-18.font-semibold.text-dark-gray',
    'class_day_rank': '#about > div.mb-10.flex.flex-row.justify-between > div:nth-child(5) > div.text-18.font-semibold.text-dark-gray',
    
    
}

In [134]:
df_first = pd.DataFrame(columns=['Name', 'Product_link', 'Post_link'])

for url in df_urls:
    if url is not np.nan:
        link = str(url).split('/')
        if len(link) > 4:
            if link[3] == 'products':
                df_first = df_first._append({'Name': link[4], 'Product_link': 'https://www.producthunt.com/' + 'products' + '/' + link[4], 'Post_link': 'https://www.producthunt.com/' + 'posts' + '/' + link[4]}, ignore_index=True)
            elif link[3] == 'posts':
                df_first = df_first._append({'Name': link[4], 'Product_link': 'https://www.producthunt.com/' + 'products' + '/' + link[4], 'Post_link': 'https://www.producthunt.com/' + 'posts' + '/' + link[4]}, ignore_index=True)
                
        else:
            df_first = df_first._append({'Product_link': np.nan, 'Post_link': np.nan}, ignore_index=True)
                
df_first

,Name,Product_link,Post_link
0,anode,https://www.producthunt.com/products/anode,https://www.producthunt.com/posts/anode
1,docask,https://www.producthunt.com/products/docask,https://www.producthunt.com/posts/docask
2,stepseed-v2,https://www.producthunt.com/products/stepseed-v2,https://www.producthunt.com/posts/stepseed-v2
3,keyzii,https://www.producthunt.com/products/keyzii,https://www.producthunt.com/posts/keyzii
4,hubql-cli,https://www.producthunt.com/products/hubql-cli,https://www.producthunt.com/posts/hubql-cli
...,...,...,...
2454,timer-plus,https://www.producthunt.com/products/timer-plus,https://www.producthunt.com/posts/timer-plus
2455,chat-plus-for-google-chat-2,https://www.producthunt.com/products/chat-plus...,https://www.producthunt.com/posts/chat-plus-fo...
2456,just-in-case-message,https://www.producthunt.com/products/just-in-c...,https://www.producthunt.com/posts/just-in-case...
2457,better-blog-comments-for-shopify,https://www.producthunt.com/products/better-bl...,https://www.producthunt.com/posts/better-blog-...


In [142]:
final_data = []


class Product:
    def __init__(self, url: str, attr: dict, headers: dict):
        self.url = url
        self.attr = attr
        self.web_obj = requests.get(url, headers=headers)
        self.web_page = BeautifulSoup(self.web_obj.text, "html.parser")
        
    def scrap_page(self):
        product_title = self.web_page.find(self.attr["product_title"], attrs={'class': self.attr['class_title']}).text
        team_member = self.web_page.find_all(self.attr['team'], attrs={'class': self.attr['class_team']})
        team_member = str(team_member)
        team_member = team_member.split(',')
        for i in range(0, len(team_member)):
            match = re.search(r'href="([^"]+)"', team_member[i])
            if match:
                href_content = match.group(1)
                team_member_link = "https://www.producthunt.com" + href_content
                team_member[i] = team_member_link
                
        return product_title, team_member

class Post:
    def __init__(self, url: str, attr: dict, headers: dict):
        self.url = url
        self.attr = attr
        self.web_obj = requests.get(url, headers=headers)
        self.web_page = BeautifulSoup(self.web_obj.text, "html.parser")
        print(self.web_page)
        
    def scrap_page(self):
        product_categories_elements = self.web_page.find_all(self.attr['product_category'], attrs={'class': self.attr['class_category']})
        product_categories = [element.text for element in product_categories_elements]
        total_up_votes = self.web_page.find(self.attr['total_up_votes'], attrs={'class': self.attr['class_up_votes']})
        print(total_up_votes)
        total_comments = self.web_page.select_one(self.attr['class_comments'])
        day_rank = self.web_page.select_one(self.attr['class_day_rank'])
        # hunter_use = 
        # maker_comments
        
        return product_categories, total_up_votes, total_comments, day_rank
        
        
post = 'https://www.producthunt.com/products/anode'
post_obj = Post(url=post, attr=attr, headers=headers)
final_data.append(post_obj.scrap_page())
''' product = 'https://www.producthunt.com/products/anode'
product_obj = Product(url=product, attr=attr, headers=headers)
final_data.append(product_obj.scrap_page())
'''
print(final_data)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><title> Anode - Product Information, Latest Updates, and Reviews 2024 | Product Hunt</title><link href="http://www.producthunt.com/products/anode" rel="canonical"/><meta content="Are you struggling with inaccurate and unreliable data? We know how you feel, that's why we built Anode, your data quality copilot! Powered by GPT-4 and modern AI tech, Anode not only detects issues but also explains the reason. Try Anode now!" name="description"/><meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/><meta content="1467820943460899" property="fb:app_id"/><meta content="Jacob Kurian" property="author"/><link href="http://www.producthunt.com/@jacob_kurian" rel="author"/><meta content="summary_large_image" name="twitter:card"/><meta content="@producthunt" name="twitter:site"/><meta content=" Anode - Product Information, Latest Updates, and Reviews 2024 | Product Hunt" name="twitter:title"/><meta content="